# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Memuat dataset dari hasil clustering
df = pd.read_csv('https://raw.githubusercontent.com/harisman7/submission-akhir-bmlp/refs/heads/main/result_df.csv')
print(df.head())

   Temperature   Humidity  Wind_Speed  Cloud_Cover     Pressure  Rain  \
0    23.720338  89.592641    7.335604    50.501694  1032.378759     1   
1    27.879734  46.489704    5.952484     4.990053   992.614189     0   
2    25.069084  83.072843    1.371992    14.855784  1007.231620     0   
3    23.622080  74.367758    7.050551    67.255282   982.632013     1   
4    20.591370  96.858822    4.643921    47.676444   980.825142     0   

   Cluster  cluster  
0        1        2  
1        3        0  
2        3        0  
3        1        1  
4        3        2  


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
# Memisahkan fitur dan target
X = df.drop('cluster', axis=1)
y = df['cluster']

# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standarisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Menampilkan dimensi data
print(f"Dimensi X_train: {X_train.shape}")
print(f"Dimensi X_test: {X_test.shape}")

Dimensi X_train: (2000, 7)
Dimensi X_test: (500, 7)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [4]:
# Inisialisasi model
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(random_state=42),
    "KNN": KNeighborsClassifier()
}

# Melatih model dan melakukan prediksi
predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions[name] = model.predict(X_test)

# Mengambil hasil prediksi
y_pred_log_reg = predictions["Logistic Regression"]
y_pred_rf = predictions["Random Forest"]
y_pred_dt = predictions["Decision Tree"]
y_pred_svm = predictions["SVM"]
y_pred_knn = predictions["KNN"]

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [5]:
# Fungsi untuk evaluasi model
def evaluate_model(y_test, y_pred, model_name):
    print(f"\nEvaluasi {model_name} sebelum tuning:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

# Evaluasi masing-masing model
for name, y_pred in zip(
    ["Logistic Regression", "Random Forest", "Decision Tree", "SVM", "K-Nearest Neighbors"],
    [y_pred_log_reg, y_pred_rf, y_pred_dt, y_pred_svm, y_pred_knn]
):
    evaluate_model(y_test, y_pred, name)


Evaluasi Logistic Regression sebelum tuning:
Accuracy: 0.9740
F1-Score: 0.9740
Confusion Matrix:
 [[154   0   2   0]
 [  0 103   2   1]
 [  3   1 115   0]
 [  0   4   0 115]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       156
           1       0.95      0.97      0.96       106
           2       0.97      0.97      0.97       119
           3       0.99      0.97      0.98       119

    accuracy                           0.97       500
   macro avg       0.97      0.97      0.97       500
weighted avg       0.97      0.97      0.97       500


Evaluasi Random Forest sebelum tuning:
Accuracy: 1.0000
F1-Score: 1.0000
Confusion Matrix:
 [[156   0   0   0]
 [  0 106   0   0]
 [  0   0 119   0]
 [  0   0   0 119]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       156
           1       1.00      1.00      1.00       106
           2       1.00      1.00      1.00       119
  

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [7]:
# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
# Fungsi untuk tuning model
def tune_model(model, param_grid, X_train, y_train, skf):
    grid_search = GridSearchCV(model, param_grid, cv=skf, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

# Inisialisasi StratifiedKFold untuk cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Tuning untuk masing-masing model
param_grid_log_reg = {'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}
best_log_reg = tune_model(LogisticRegression(random_state=42, max_iter=2000, solver='liblinear'), param_grid_log_reg, X_train, y_train, skf)

param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
best_rf_clf = tune_model(RandomForestClassifier(random_state=42), param_grid_rf, X_train, y_train, skf)

param_grid_dt = {'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
best_dt_clf = tune_model(DecisionTreeClassifier(random_state=42), param_grid_dt, X_train, y_train, skf)

param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
best_svm_clf = tune_model(SVC(random_state=42), param_grid_svm, X_train, y_train, skf)

param_grid_knn = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
best_knn_clf = tune_model(KNeighborsClassifier(), param_grid_knn, X_train, y_train, skf)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [9]:
# Fungsi untuk evaluasi model setelah tuning
def evaluate_tuned_model(y_test, best_model, model_name):
    y_pred_best = best_model.predict(X_test)
    print(f"\nEvaluasi {model_name} setelah tuning:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_best, average='weighted'):.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best))
    print(classification_report(y_test, y_pred_best))

# Evaluasi model setelah tuning menggunakan loop
models = {
    "Logistic Regression": best_log_reg,
    "Random Forest": best_rf_clf,
    "Decision Tree": best_dt_clf,
    "SVM": best_svm_clf,
    "K-Nearest Neighbors": best_knn_clf
}

for model_name, best_model in models.items():
    evaluate_tuned_model(y_test, best_model, model_name)


Evaluasi Logistic Regression setelah tuning:
Accuracy: 0.9260
F1-Score: 0.9249
Confusion Matrix:
 [[156   0   0   0]
 [  0  80  25   1]
 [  3   8 108   0]
 [  0   0   0 119]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       156
           1       0.91      0.75      0.82       106
           2       0.81      0.91      0.86       119
           3       0.99      1.00      1.00       119

    accuracy                           0.93       500
   macro avg       0.92      0.92      0.92       500
weighted avg       0.93      0.93      0.92       500


Evaluasi Random Forest setelah tuning:
Accuracy: 1.0000
F1-Score: 1.0000
Confusion Matrix:
 [[156   0   0   0]
 [  0 106   0   0]
 [  0   0 119   0]
 [  0   0   0 119]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       156
           1       1.00      1.00      1.00       106
           2       1.00      1.00      1.00       119
  

In [10]:
# Fungsi untuk cross-validation performance
def cross_val_performance(model, X, y, skf):
    cross_val_accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    cross_val_f1 = cross_val_score(model, X, y, cv=skf, scoring='f1_weighted')
    return cross_val_accuracy, cross_val_f1

# Menyusun model dan nama model dalam dictionary
models = {
    "Logistic Regression": best_log_reg,
    "Random Forest": best_rf_clf,
    "Decision Tree": best_dt_clf,
    "SVM": best_svm_clf,
    "K-Nearest Neighbors": best_knn_clf
}

# Cross-validation untuk masing-masing model
for name, model in models.items():
    cross_val_accuracy, cross_val_f1 = cross_val_performance(model, X, y, skf)
    print(f"\nCross-validation untuk {name}:")
    print(f"Accuracy: {cross_val_accuracy.mean():.4f} ± {cross_val_accuracy.std():.4f}")
    print(f"F1-Score: {cross_val_f1.mean():.4f} ± {cross_val_f1.std():.4f}")


Cross-validation untuk Logistic Regression:
Accuracy: 0.9008 ± 0.0102
F1-Score: 0.8981 ± 0.0106

Cross-validation untuk Random Forest:
Accuracy: 0.9964 ± 0.0015
F1-Score: 0.9964 ± 0.0015

Cross-validation untuk Decision Tree:
Accuracy: 0.9988 ± 0.0010
F1-Score: 0.9988 ± 0.0010

Cross-validation untuk SVM:
Accuracy: 0.9932 ± 0.0053
F1-Score: 0.9932 ± 0.0053

Cross-validation untuk K-Nearest Neighbors:
Accuracy: 0.9320 ± 0.0110
F1-Score: 0.9319 ± 0.0111


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

Berdasarkan hasil evaluasi model (Logistic Regression, Random Forest, Decision Tree, dan SVM), berikut adalah analisis dan rekomendasi tindakan lanjutan:
#### 1. Perbandingan Hasil Evaluasi
- **Logistic Regression**: Akurasi 0.9008, F1-Score 0.8981
- **Random Forest**: Akurasi 0.9964, F1-Score 0.9964
- **Decision Tree**: Akurasi 0.9988, F1-Score 0.9988
- **SVM**: Akurasi 0.9932, F1-Score 0.9932

Secara keseluruhan, **Random Forest** dan **Decision Tree** menunjukkan performa terbaik dengan akurasi serta F1-Score yang sangat tinggi dan konsisten. Sementara itu, **Logistic Regression** dan **SVM** juga memberikan hasil yang baik, meskipun sedikit lebih rendah.

#### 2. Identifikasi Kelemahan Model
- ### **Precision dan Recall**  
Berdasarkan hasil evaluasi, **Logistic Regression** menunjukkan F1-Score 0.8981, yang mengindikasikan kemungkinan adanya ketidakseimbangan antara Precision dan Recall, terutama jika data memiliki karakteristik yang kompleks atau tidak linier. Pada **Random Forest** dan **Decision Tree**, F1-Score yang sangat tinggi (0.9964 dan 0.9988) menunjukkan performa Precision dan Recall yang hampir sempurna, tetapi perlu analisis lebih lanjut untuk memastikan tidak ada dominasi salah satu metrik. Sementara itu, **SVM** dengan F1-Score 0.9932 juga menunjukkan keseimbangan Precision dan Recall yang baik, meskipun performanya sedikit di bawah Random Forest dan Decision Tree, kemungkinan karena kurang optimalnya hyperparameter atau batas margin.

- ### **Overfitting atau Underfitting**  
Berdasarkan hasil evaluasi, **Logistic Regression** cenderung mengalami **underfitting**, terutama karena model ini bersifat linear dan mungkin kurang mampu menangkap pola non-linear dalam data. Sebaliknya, **Random Forest** dan **Decision Tree** memiliki risiko **overfitting** yang tinggi, terutama Decision Tree, karena model ini dapat dengan mudah menyesuaikan diri dengan data pelatihan hingga menghasilkan akurasi dan F1-Score hampir sempurna. **SVM** berada di antara kedua ekstrem ini, dengan potensi **underfitting** jika parameter seperti kernel atau margin tidak diatur dengan baik, tetapi juga dapat mengalami **overfitting** jika menangani dataset dengan jumlah fitur yang sangat besar tanpa regularisasi.

#### 3. Rekomendasi Tindakan Lanjutan
- ### **1. Analisis Confusion Matrix**  
Confusion matrix penting untuk memahami distribusi prediksi model, khususnya kesalahan tipe I (false positives) dan tipe II (false negatives). Analisis ini membantu menentukan prioritas antara Precision dan Recall sesuai kebutuhan aplikasi, misalnya mengurangi false negatives untuk aplikasi yang memerlukan sensitivitas tinggi.

- ### **2. Validasi dengan Data Baru**  
Validasi menggunakan **k-fold cross-validation** dan dataset terpisah (test set) memastikan performa model tidak hanya baik di data pelatihan tetapi juga dapat diandalkan di dunia nyata. Langkah ini penting untuk mengidentifikasi kemampuan generalisasi model.

- ### **3. Pengurangan Overfitting**  
Overfitting pada Decision Tree dapat diatasi dengan pruning, sedangkan Random Forest dapat dioptimalkan melalui pengaturan jumlah pohon (**n_estimators**) dan kedalaman maksimum pohon (**max_depth**). Langkah ini memastikan model tetap efisien dan tidak hanya menghafal data.

- ### **4. Hyperparameter Tuning**  
Menggunakan **Grid Search** atau **Random Search** untuk optimasi hyperparameter seperti **C** dan **gamma** pada SVM, atau kedalaman pohon pada Random Forest, membantu meningkatkan performa model. Tuning ini memastikan model dapat menangani data kompleks dengan lebih baik.